In [1]:
from transformers import AutoTokenizer
from transformers import LlamaForCausalLM
import torch

device = torch.device("cuda")
ckpt = '/opt/Data/ModelWeight/IDEA-CCNL/Ziya-LLaMA-13B'

model = LlamaForCausalLM.from_pretrained(ckpt, torch_dtype=torch.float16, device_map="cuda:0", load_in_8bit=True)
# model = LlamaForCausalLM.from_pretrained(ckpt, torch_dtype=torch.float16, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=False)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [2]:
query="帮我写一份去西安的旅游计划"
inputs = '<human>:' + query.strip() + '\n<bot>:'
      
input_ids = tokenizer(inputs, return_tensors="pt").input_ids.to(device)
generate_ids = model.generate(
            input_ids,
            max_new_tokens=1024, 
            do_sample = True, 
            top_p = 0.85, 
            temperature = 1.0, 
            repetition_penalty=1., 
            eos_token_id=2, 
            bos_token_id=1, 
            pad_token_id=0)
output = tokenizer.batch_decode(generate_ids)[0]
print(output)

<s> <human> :帮我写一份去西安的旅游计划 <bot>  : 当然，我可以帮你写一份西安旅游计划。请问你需要多长时间的旅游计划？</s>


In [4]:
query="你好，帮我介绍一下南京市，详细内容字数100"
inputs = '<human>:' + query.strip() + '\n<bot>:'
      
input_ids = tokenizer(inputs, return_tensors="pt").input_ids.to(device)
generate_ids = model.generate(
            input_ids,
            max_new_tokens=1024, 
            do_sample = True, 
            top_p = 0.85, 
            temperature = 1.0, 
            repetition_penalty=1., 
            eos_token_id=2, 
            bos_token_id=1, 
            pad_token_id=0)
output = tokenizer.batch_decode(generate_ids)[0]
print(output)

<s> <human> :你好，帮我介绍一下南京市，详细内容字数100 <bot>  : 南京市是中国江苏省的省会，位于长江下游，是一个具有悠久历史和丰富文化的城市。南京市拥有丰富的人文遗产，包括中山陵、明孝陵、总统府等著名景点。此外，南京市还以美食闻名，著名的美食包括南京盐水鸭、鸭血粉丝汤等。南京市的气候温和，四季分明，是一个适宜居住和旅游的城市。</s>
